In [29]:
import pandas as pd
import regex as re
import requests
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
cdd_url = "http://cdd.publicsafety.gc.ca/dl-eng.aspx?cultureCode=en-Ca&normalizedCostYear=1"

# Issue with directly loading it as DataFrame

In [5]:
df = pd.read_csv(cdd_url, delimiter="\t")

In [6]:
df

,EVENT CATEGORY,EVENT GROUP,EVENT SUBGROUP,EVENT TYPE,PLACE,EVENT START DATE,COMMENTS,FATALITIES,INJURED / INFECTED,EVACUATED,...,FEDERAL DFAA PAYMENTS,PROVINCIAL DFAA PAYMENTS,PROVINCIAL DEPARTMENT PAYMENTS,MUNICIPAL COSTS,OGD COSTS,INSURANCE PAYMENTS,NGO PAYMENTS,UTILITY - PEOPLE AFFECTED,MAGNITUDE,Unnamed: 22
0,Incident,Technology,Fire,Non-Residential,Ottawa ON and Hull QC,4/26/1900 12:00:00 AM,"Ottawa ON and Hull QC, April 26, 1900. A smal...",7.0,0.0,15000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,Incident,Technology,Explosion,Non-Residential,Grand Forks BC,1/1/1901 12:00:00 AM,"Grand Forks BC, 1901. An explosion in the Unio...",64.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,Incident,Technology,Infrastructure failure,Manufacturing / Industry,Wellington BC,1/1/1901 12:00:00 AM,"Wellington BC, 1901. Sixteen people were entom...",16.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
3,Incident,Technology,Explosion,Non-Residential,"Extension, British Columbia",10/5/1909 12:00:00 AM,"On the morning of October 5th, an explosion an...",32.0,25.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,Disaster,Natural,Meteorological - Hydrological,Flood,Saint John River Basin NB,3/1/1902 12:00:00 AM,"Saint John River Basin NB, March 1902. An unus...",2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,Incident,Technology,Fire,Non-Residential,"Whitehorse, Yukon",5/23/1905 12:00:00 AM,"On the morning of May 23rd, a fire broke out i...",0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1320,Disaster,Natural,Meteorological - Hydrological,Hurricane / Typhoon / Tropical Storm,Eastern Canada,9/11/1900 12:00:00 AM,Remnants of the Great Galveston Hurricane cros...,54.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1321,Incident,Technology,Transportation accident,Marine,"Off Cappahayden, Newfoundland",2/24/1918 12:00:00 AM,"The ""SS Florizel"" was a Red Cross Line passeng...",94.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1322,Incident,Technology,Transportation accident,Marine,Lake Superior,11/21/1902 12:00:00 AM,"The 75 m (245 ft) canal steamer the ""Bannockbu...",21.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [17]:
# Check how many distinct values in the "EVENT CATEGORY" column
df["EVENT CATEGORY"].nunique()

25

In [18]:
# Check what are the distinct values in the "EVENT CATEGORY" column
df["EVENT CATEGORY"].unique()

array(['Incident', 'Disaster',
       'Note: Federal and Provincial payments are interim and do not represent final payment totals.',
       'Response and recovery activities began at the time of the event. The ice storm significantly impacted some of Canada?s largest and most densely populated urban municipalities. Municipalities, provincial ministries and non-governmental organizations such as the Red Cross all acted to respond to the ice storm. Ontario determined that the impacts of the storm were severe enough to warrant implementation of an Ice Storm Assistance Program to help municipalities and conservation authorities with emergency response and recovery costs. Two deaths were attributed to the storm due to carbon monoxide poisoning from improperly ventilated generators and heaters.  ',
       '*Note: A request for federal assistance through the DFAA program has been made; however, no payments have yet been issued.',
       '*Note: A request for federal assistance through the DF

# Solution for getting rid of incorrect return

In [20]:
# Read the data as plain text rather than DataFrame (hint: use requests)
req = requests.get(cdd_url)
data=req.text

In [21]:
# Ctrl + F to check a few examples in the EVENT CATEGORY column, and find the pattern 
data

'EVENT CATEGORY\tEVENT GROUP\tEVENT SUBGROUP\tEVENT TYPE\tPLACE\tEVENT START DATE\tCOMMENTS\tFATALITIES\tINJURED / INFECTED\tEVACUATED\tESTIMATED TOTAL COST\tNORMALIZED TOTAL COST\tEVENT END DATE\tFEDERAL DFAA PAYMENTS\tPROVINCIAL DFAA PAYMENTS\tPROVINCIAL DEPARTMENT PAYMENTS\tMUNICIPAL COSTS\tOGD COSTS\tINSURANCE PAYMENTS\tNGO PAYMENTS\tUTILITY - PEOPLE AFFECTED\tMAGNITUDE\t\r\nIncident\tTechnology\tFire\tNon-Residential\tOttawa ON and Hull QC\t4/26/1900 12:00:00 AM\tOttawa ON and Hull QC, April 26, 1900.  A small chimney fire in Hull quickly spread to neighbouring buildings and then engulfed the Eddy lumberyard.  A strong wind carried the fire to the Ottawa side.  By the time the fire ended, 3000 buildings had been destroyed in the Ottawa-Hull region.\t7\t0\t15000\t\t\t4/26/1900 12:00:00 AM\t\t\t\t\t\t\t\t0\t0.0\t\r\nIncident\tTechnology\tExplosion\tNon-Residential\tGrand Forks BC\t1/1/1901 12:00:00 AM\tGrand Forks BC, 1901. An explosion in the Union Mine killed 64 people.\t64\t0\t0\

In [24]:
# Find all the cases where the similar pattern are observed in the text (hint: use regex)
# Very useful link: https://www.w3schools.com/python/python_regex.asp
data_now = re.sub(r"\.\s*\n+",". ",data)

In [25]:
data_now

'EVENT CATEGORY\tEVENT GROUP\tEVENT SUBGROUP\tEVENT TYPE\tPLACE\tEVENT START DATE\tCOMMENTS\tFATALITIES\tINJURED / INFECTED\tEVACUATED\tESTIMATED TOTAL COST\tNORMALIZED TOTAL COST\tEVENT END DATE\tFEDERAL DFAA PAYMENTS\tPROVINCIAL DFAA PAYMENTS\tPROVINCIAL DEPARTMENT PAYMENTS\tMUNICIPAL COSTS\tOGD COSTS\tINSURANCE PAYMENTS\tNGO PAYMENTS\tUTILITY - PEOPLE AFFECTED\tMAGNITUDE\t\r\nIncident\tTechnology\tFire\tNon-Residential\tOttawa ON and Hull QC\t4/26/1900 12:00:00 AM\tOttawa ON and Hull QC, April 26, 1900.  A small chimney fire in Hull quickly spread to neighbouring buildings and then engulfed the Eddy lumberyard.  A strong wind carried the fire to the Ottawa side.  By the time the fire ended, 3000 buildings had been destroyed in the Ottawa-Hull region.\t7\t0\t15000\t\t\t4/26/1900 12:00:00 AM\t\t\t\t\t\t\t\t0\t0.0\t\r\nIncident\tTechnology\tExplosion\tNon-Residential\tGrand Forks BC\t1/1/1901 12:00:00 AM\tGrand Forks BC, 1901. An explosion in the Union Mine killed 64 people.\t64\t0\t0\

In [31]:
# Get rid of the carriage return
df = pd.read_csv(StringIO(data_now), delimiter = "\t")

In [ ]:
# Create dataframe from the new edited text


In [32]:
# Check how many distinct values in the "EVENT CATEGORY" column
df["EVENT CATEGORY"].nunique()

2

In [33]:
# Check what are the distinct values in the "EVENT CATEGORY" column
df["EVENT CATEGORY"].unique()

array(['Incident', 'Disaster'], dtype=object)

# Make it as a script

In [35]:
import pandas as pd
import regex as re
import requests
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
cdd_url = "http://cdd.publicsafety.gc.ca/dl-eng.aspx?cultureCode=en-Ca&normalizedCostYear=1"
    
req = requests.get(cdd_url)
data=req.text    
data_now = re.sub(r"\.\s*\n+",". ",data)
df = pd.read_csv(StringIO(data_now), delimiter = "\t")
df["EVENT CATEGORY"].nunique()
df["EVENT CATEGORY"].unique()

array(['Incident', 'Disaster'], dtype=object)